In [1]:
# -*- coding: utf-8 -*-
import keras
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss

from load_cifar10 import load_cifar10_data
from load_pascal2012 import load_pascal2012_data
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K
K.set_image_dim_ordering('th')
import sklearn.metrics as skm

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

Using Theano backend.


In [2]:
def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):
    """VGG 16 Model for Keras

    Model Schema is based on 
    https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3

    ImageNet Pretrained Weights 
    https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view?usp=sharing

    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of categories for our classification task
    """
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(channel, img_rows, img_cols)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Add Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    # Loads ImageNet pre-trained data
    model.load_weights('imagenet_models/vgg16_weights_th_dim_ordering_th_kernels.h5')

    # Truncate and replace softmax layer for transfer learning
    model.layers.pop()
    for layer in model.layers: layer.trainable = False
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    model.add(Dense(num_classes, activation='sigmoid'))

    # Uncomment below to set the first 10 layers to non-trainable (weights will not be updated)
    #for layer in model.layers[:10]:
    #    layer.trainable = False

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-5, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd,
                  loss='binary_crossentropy',
                  metrics=['accuracy',precision, recall, f1])

    return model

In [3]:
def precision(y_true, y_pred):
    """Precision metric.		

    Only computes a batch-wise average of precision.		

    Computes the precision, a metric for multi-label classification of		
    how many selected items are relevant.		
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.		

    Only computes a batch-wise average of recall.		

    Computes the recall, a metric for multi-label classification of		
    how many relevant items are selected.		
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    pre = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((pre*rec)/(pre+rec))

In [4]:
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 20
batch_size = 16 
nb_epoch = 10

# Load Cifar10 data. Please implement your own load_data() module for your own dataset
# X_train, Y_train, X_valid, Y_valid = load_cifar10_data(img_rows, img_cols)
X_train, Y_train, X_valid, Y_valid = load_pascal2012_data(img_rows, img_cols)

# Load our model
model = vgg16_model(img_rows, img_cols, channel, num_classes)

['aeroplane_train.txt', 'bicycle_train.txt', 'bird_train.txt', 'boat_train.txt', 'bottle_train.txt', 'bus_train.txt', 'car_train.txt', 'cat_train.txt', 'chair_train.txt', 'cow_train.txt', 'diningtable_train.txt', 'dog_train.txt', 'horse_train.txt', 'motorbike_train.txt', 'person_train.txt', 'pottedplant_train.txt', 'sheep_train.txt', 'sofa_train.txt', 'train_train.txt', 'tvmonitor_train.txt']
0: aeroplane_train.txt

1: bicycle_train.txt

2: bird_train.txt

3: boat_train.txt

4: bottle_train.txt

5: bus_train.txt

6: car_train.txt

7: cat_train.txt

8: chair_train.txt

9: cow_train.txt

10: diningtable_train.txt

11: dog_train.txt

12: horse_train.txt

13: motorbike_train.txt

14: person_train.txt

15: pottedplant_train.txt

16: sheep_train.txt

17: sofa_train.txt

18: train_train.txt

19: tvmonitor_train.txt



VOC2012/JPEGImages/2008_000033.jpg [0]aeroplane_train.txt
VOC2012/JPEGImages/2008_000037.jpg [0]aeroplane_train.txt
VOC2012/JPEGImages/2008_000151.jpg [0]aeroplane_train.txt
VOC2012/JPEGImages/2008_000197.jpg [0]aeroplane_train.txt
VOC2012/JPEGImages/2008_000291.jpg [0]aeroplane_train.txt
VOC2012/JPEGImages/2008_000095.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000097.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000103.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000131.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000192.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000318.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000350.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000361.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000400.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000512.jpg [2]bird_train.txt
VOC2012/JPEGImages/2008_000148.jpg [3]boat_train.txt
VOC2012/JPEGImages/2008_000262.jpg [3]boat_train.txt
VOC2012/JPEGImages/2008_000405.jpg [3]boat_train.txt
VOC2012/JPEGImages/20

19: tvmonitor_val.txt



VOC2012/JPEGImages/2008_000021.jpg [0]aeroplane_val.txt
VOC2012/JPEGImages/2008_000064.jpg [0]aeroplane_val.txt
VOC2012/JPEGImages/2008_000466.jpg [1]bicycle_val.txt
VOC2012/JPEGImages/2008_000054.jpg [2]bird_val.txt
VOC2012/JPEGImages/2008_000123.jpg [2]bird_val.txt
VOC2012/JPEGImages/2008_000134.jpg [2]bird_val.txt
VOC2012/JPEGImages/2008_000339.jpg [2]bird_val.txt
VOC2012/JPEGImages/2008_000472.jpg [2]bird_val.txt
VOC2012/JPEGImages/2008_000007.jpg [3]boat_val.txt
VOC2012/JPEGImages/2008_000120.jpg [3]boat_val.txt
VOC2012/JPEGImages/2008_000140.jpg [3]boat_val.txt
VOC2012/JPEGImages/2008_000414.jpg [3]boat_val.txt
VOC2012/JPEGImages/2008_000272.jpg [4]bottle_val.txt
VOC2012/JPEGImages/2008_000075.jpg [5]bus_val.txt
VOC2012/JPEGImages/2008_000027.jpg [6]car_val.txt
VOC2012/JPEGImages/2008_000042.jpg [6]car_val.txt
VOC2012/JPEGImages/2008_000050.jpg [6]car_val.txt
VOC2012/JPEGImages/2008_000163.jpg [6]car_val.txt
VOC2012/JPEGImages/2008_000174.jpg [6]car_val.txt
VOC2012/JPEGImages/200

/Users/qianminming/keras/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


...

/Users/qianminming/keras/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`


/Users/qianminming/keras/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`


/Users/qianminming/keras/lib/python3.6/site-packages/ipykernel_launcher.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`


/Users/qianminming/keras/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`


/Users/qianminming/keras/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
